In [18]:
from scipy.io import wavfile as wv
import scipy.signal as ss
from scipy.stats import entropy
import numpy as np
import os
import librosa
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FastICA
from sklearn.cluster import KMeans

In [2]:
# Directory to my training set should change
directory = "/Users/noahvilla/Downloads/Dataset/train"

audio_data_list = []
sampling_rate_list = []

# Iterate over training data
for filename in os.listdir(directory):
    # Loads all the .wav files in the traning set
    if filename.endswith(".wav"):
        file_path = os.path.join(directory, filename)
        audio_data, sampling_rate = librosa.load(file_path, sr=None)
        
        # Append the audio data and sampling rate to the lists
        audio_data_list.append(audio_data)
        sampling_rate_list.append(sampling_rate)

# Now we can use the audio data and sampling rate data

In [3]:
#Gathers the frequency information of a song
fft_list = []
for song in audio_data_list:
    fft_list.append(np.fft.fft(song))

In [4]:
# Computes energy of each song
energy_list = []
for song in audio_data_list:
    energy_list.append(np.sum(np.square(song)))

In [5]:
# Finds the variance and mean of each song
variance_list = []
expectation_list = []
for song in audio_data_list:
    variance_list.append(np.var(song))
    expectation_list.append(np.mean(song))

In [9]:
tempo_list = []
for i in range(0,len(audio_data_list)):
    tempo, _ = librosa.beat.beat_track(y=audio_data_list[i], sr=sampling_rate_list[i])
    tempo_list.append(tempo)

In [16]:
zero_cross_rate_list = []
for song in audio_data_list:
    zero_cross_rate_list.append(np.median(librosa.feature.zero_crossing_rate(song)))

-inf


In [ ]:
fft_entrp_list = []
for fft in fft_list:
    fft_entrp_list.append(entropy(np.absolute(fft)))

In [ ]:
def conv_compare(song, data):
    """
    Sees how similar the fft of the songs are, higher number means more overlap
    song is one song and data is all the songs to compare song to
    can be used for time or for freq domain info
    """
    flipped_song = np.flip(song)
    convolution_sim_list = []
    for audio in data:
        sum = 0
        for i in range(0,len(audio)):
            sum += audio[i]*flipped_song[i]
        convolution_sim_list.append(sum)

In [ ]:
# Make the Feature Matrix
feature_matrix = np.column_stack((energy_list, variance_list, expectation_list, tempo_list, zero_cross_rate_list, fft_entrp_list))

# Normalize
scaler = StandardScaler()
normalized_features = scaler.fit_transform(feature_matrix)

# Apply ICA
ica = FastICA(n_components=10, random_state=42)
independent_components = ica.fit_transform(normalized_features)

# Cluster
kmeans = KMeans(n_clusters=10, random_state=42)
clusters = kmeans.fit_predict(independent_components)

# Step 5: Evaluation
# You may need to manually inspect the clusters to see if they align with the known genres

# Assign genres to clusters (these clusters are just groups, the labels may not be correct, should listen to some of these songs)
genre_mapping = {
    0: 'blues',
    1: 'classical',
    8: 'country',
    3: 'disco',
    4: 'hiphop',
    5: 'jazz',
    6: 'metal',
    7: 'pop',
    2: 'reggae',
    9: 'rock'
}

# Map cluster labels to genres
predicted_genres = [genre_mapping[cluster] for cluster in clusters]

# Print predicted genres for each song in the required format
print("ID,Genre")
for filename, genre in zip(os.listdir(directory), predicted_genres):
    print(f"{filename},{genre}")

In [24]:
import pandas as pd
df = pd.DataFrame({'ID': os.listdir('/Users/noahvilla/Downloads/Dataset'), 'genre': predicted_genres})
df.to_csv('prediction.csv', index=False)

NameError: name 'predicted_genres' is not defined